In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from pathlib import Path
from IPython.core.debugger import set_trace
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import operator
from pathlib import Path
import numpy as np
from torch import nn


import torch.nn.functional as F

from torch.nn import init

def normalize(x, m, s): return (x-m)/s

def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"
    
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"
    
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)

def test_near(a,b): test(a,b,near)

def test_eq(a,b): test(a,b,operator.eq,'==')

def get_data():
    path = Path('./mnist.pkl.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))



## Initial setup

### Data

In [3]:
mpl.rcParams['image.cmap'] = 'gray'

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [6]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [7]:
model = Model(m, nh, 10)

In [8]:
pred = model(x_train)

### Cross entropy loss

First, we will need to compute the softmax of our activations. This is defined by:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

or more concisely:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum_{0 \leq j \leq n-1} e^{x_{j}}}$$ 

In practice, we will need the log of the softmax when we calculate the loss.

In [9]:
pred[2]

tensor([-0.1521,  0.0866,  0.0043,  0.0747, -0.0500, -0.0599,  0.0412,  0.1390,
        -0.0071,  0.0086], grad_fn=<SelectBackward>)

In [10]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

In [11]:
sm_pred = log_softmax(pred)

The cross entropy loss for some target $x$ and some prediction $p(x)$ is given by:

$$ -\sum x\, \log p(x) $$

But since our $x$s are 1-hot encoded, this can be rewritten as $-\log(p_{i})$ where i is the index of the desired target.

This can be done using numpy-style [integer array indexing](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html#integer-array-indexing). Note that PyTorch supports all the tricks in the advanced indexing methods discussed in that link.

In [12]:
y_train[:3]

tensor([5, 0, 4])

In [13]:
sm_pred[[0,1,2], [5,0,4]]

tensor([-2.2645, -2.4393, -2.3642], grad_fn=<IndexBackward>)

In [14]:
y_train.shape[0]

50000

In [15]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()


In [16]:
loss = nll(sm_pred, y_train)

In [17]:
loss

tensor(2.3049, grad_fn=<NegBackward>)

Note that the formula 

$$\log \left ( \frac{a}{b} \right ) = \log(a) - \log(b)$$ 

gives a simplification when we compute the log softmax, which was previously defined as `(x.exp()/(x.exp().sum(-1,keepdim=True))).log()`

In [18]:
def log_softmax(x): return x - x.exp().sum(-1,keepdim=True).log()

In [19]:
# log연산법칙을 이용해서 식을 바꿔도 결과가 같음

test_near(nll(log_softmax(pred), y_train), loss)

Then, there is a way to compute the log of the sum of exponentials in a more stable way, called the [LogSumExp trick](https://en.wikipedia.org/wiki/LogSumExp). The idea is to use the following formula:

$$\log \left ( \sum_{j=1}^{n} e^{x_{j}} \right ) = \log \left ( e^{a} \sum_{j=1}^{n} e^{x_{j}-a} \right ) = a + \log \left ( \sum_{j=1}^{n} e^{x_{j}-a} \right )$$

where a is the maximum of the $x_{j}$.

In [20]:
def logsumexp(x):

    m = x.max(-1)[0] # return (value, index) , value - 각 row에서의 최댓값
    return m + (x-m[:,None]).exp().sum(-1).log()

This way, we will avoid an overflow when taking the exponential of a big activation. In PyTorch, this is already implemented for us. 

In [21]:
test_near(logsumexp(pred), pred.logsumexp(-1))

So we can use it for our `log_softmax` function.

In [22]:
def log_softmax(x): return x - x.logsumexp(-1,keepdim=True)

In [23]:
test_near(nll(log_softmax(pred), y_train), loss)

Then use PyTorch's implementation.

In [24]:
test_near(F.nll_loss(F.log_softmax(pred, -1), y_train), loss)

In PyTorch, `F.log_softmax` and `F.nll_loss` are combined in one optimized function, `F.cross_entropy`.

In [25]:
test_near(F.cross_entropy(pred, y_train), loss)

## Basic training loop

Basically the training loop repeats over the following steps:
- get the output of the model on a batch of inputs
- compare the output to the labels we have and compute a loss
- calculate the gradients of the loss with respect to every parameter of the model
- update said parameters with those gradients to make them a little bit better

In [26]:
loss_func = F.cross_entropy

In [27]:
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

In [28]:
bs=64                  # batch size
xb = x_train[0:bs]     # a mini-batch from x
preds = model(xb)      # predictions
preds[0], preds.shape

(tensor([-0.0443,  0.0633, -0.0100,  0.0621,  0.0206,  0.0952,  0.0346,  0.1900,
          0.1360,  0.0017], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [29]:
yb = y_train[0:bs]
loss_func(preds, yb)

tensor(2.3145, grad_fn=<NllLossBackward>)

In [30]:
accuracy(preds, yb)

tensor(0.1094)

In [31]:
lr = 0.5   # learning rate
epochs = 1 # how many epochs to train for

In [32]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = loss_func(model(xb), yb)

        loss.backward()
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias   -= l.bias.grad   * lr
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()

In [33]:
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.1406, grad_fn=<NllLossBackward>), tensor(0.9375))